Doc title: **Inventory Alarming Snippet**

Article notes: This snippet is dedicating to automate the alarming procedure of inventory data which came from [Jushuitan SaaS ERP](http://www.erp321.com) as it cannot offer a proper approach to meet the request of daily alarming regarding the shortage of inventory.

文章备注：这个小程序用于解决聚水潭ERP无法提供每日库存变化后发生商品数量短缺的预警问题。

Last modified date: 2021-03-20

In [1]:
# 全局变量定义部分

# 库存（起始日期）导出文件名 🔔 
START_FILE_NAME = '商品库存_2019-07-25_19-22-18.xlsx'

# 库存（终止日期）导出文件名 🔔 
END_FILE_NAME = '商品库存_2019-07-26_14-28-07.xlsx'

# 预警数量（小于等于）阈值
ALARM_QUANTITY = 10

# 导出csv文件名（无需后缀名）
EXPORT_ALARM_FILE_NAME = '商品库存预警报告'
EXPORT_INBOUND_FILE_NAME = '商品库存增加提示'

注：

1. 以上部分除有 🔔 标志的部分外，其他部分日常均不需改动。


2. 各部分改动后，须点击菜单项 **Cell / Run All** 重新运行一次，数据方可得到更新。

3. 退出此小程序时，谨记点击菜单项 **File / Save and Checkpoint** 或 工具栏图标 <img src='jupyter_notebook_save.png' style='display: inline; vertical-align: bottom;'> 保存文件。


4. 以下程序日常不需改动。

In [2]:
# 引入pandas数据分析模块
import pandas as pd
from datetime import date

# 定义CSS列风格函数
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'background-color: red'` for negative
    strings, green otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

# 显示预警数据及生成预警报告

In [3]:
# 读取库存（起始日期）导出文件
start_df = pd.read_excel(START_FILE_NAME, usecols=['商品编码', '数量']).query('数量 > {}'.format(ALARM_QUANTITY)).groupby('商品编码').sum().sort_index()

# 读取库存（终止日期）导出文件
end_df = pd.read_excel(END_FILE_NAME, usecols=['商品编码', '数量']).query('数量 <= {}'.format(ALARM_QUANTITY)).groupby('商品编码').sum().sort_index()

# 以交集方式合并表格
result_df = end_df.merge(start_df, how='inner', left_on='商品编码', right_on='商品编码', suffixes=('_期末', '_期初'))[['数量_期初', '数量_期末']]

# 定制列显示风格
result_df['变化量'] = result_df['数量_期末'] - result_df['数量_期初']

# 导出csv文件
# result_df.to_csv('{0}_{1}.csv'.format(EXPORT_ALARM_FILE_NAME, str(date.today())))

# 按定制风格显示
result_df.style.applymap(color_negative_red).format({'变化量': '{:+.0f}'})

,数量_期初,数量_期末,变化量
商品编码,,,
HM0720-1038,12,6,-6
HM0752-3140,23,4,-19
HM0807-3543,11,10,-1
HM0814X-3042,11,9,-2
HM0816-1843,11,2,-9
HM0823-2046,11,10,-1
HM0902-1131,11,9,-2


# 显示库存增加数据及生成提示报告

In [4]:
# 读取库存（起始日期）导出文件
start_df = pd.read_excel(START_FILE_NAME, usecols=['商品编码', '数量']).query('数量 <= {}'.format(ALARM_QUANTITY)).groupby('商品编码').sum().sort_index()

# 读取库存（终止日期）导出文件
end_df = pd.read_excel(END_FILE_NAME, usecols=['商品编码', '数量']).query('数量 > {}'.format(ALARM_QUANTITY)).groupby('商品编码').sum().sort_index()

# 以交集方式合并表格
result_df = end_df.merge(start_df, how='inner', left_on='商品编码', right_on='商品编码', suffixes=('_期末', '_期初'))[['数量_期初', '数量_期末']]

# 定制列显示风格
result_df['变化量'] = result_df['数量_期末'] - result_df['数量_期初']

# 导出csv文件
# result_df.to_csv('{0}_{1}.csv'.format(EXPORT_INBOUND_FILE_NAME, str(date.today())))

# 按定制风格显示
result_df.style.applymap(color_negative_red).format({'变化量': '{:+.0f}'})

,数量_期初,数量_期末,变化量
商品编码,,,
HM0720-1039,0,20,+20
HM0720-136,0,34,+34
HM0741-0238,0,23,+23
HM0757-237,6,29,+23
